# Aspheric lens debug

In [10]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

from cavity import *
from ipywidgets import Layout, interact, FloatSlider, Checkbox, Dropdown, Text

In [11]:
# %%



import ipywidgets as widgets
from IPython.display import display

def update_plot(phi, theta):
    f=20.0
    T_c=3.0
    n_1 = 1
    n_2 = 1.5
    polynomial_coefficients = [-5.47939897e-06, 4.54562088e-02, 4.02452659e-05, 5.53445352e-08, 6.96909906e-11] # generated for f=20, Tc=3 in aspheric_lens_generator.py
    polynomial = Polynomial(polynomial_coefficients)

    optical_axis = np.array([np.cos(phi), np.sin(phi), 0])
    
    diameter = 15
    back_center = f * optical_axis
    front_center = back_center + T_c * optical_axis
    fig, ax = plt.subplots(figsize=(15, 15))
    s_1 = FlatRefractiveSurface(outwards_normal=optical_axis, center=back_center, n_1=n_1, n_2=n_2, diameter=diameter)

    s_2 = AsphericRefractiveSurface(center=front_center,
                                    outwards_normal=optical_axis,
                                    diameter=diameter,
                                    polynomial_coefficients=polynomial,
                                    n_1=n_2,
                                    n_2=n_1)
    
    ray_initial = Ray(origin=np.array([[0, 0, 0], [0, 0, 0]]),
                      k_vector=np.array([[np.cos(-theta + phi), np.sin(-theta + phi), 0],
                                         # [np.cos(phi), np.sin(phi), 0],
                                         [np.cos(theta/2 + phi), np.sin(theta/2 + phi), 0]]))
    
    intersection_1 = s_1.find_intersection_with_ray_exact(ray_initial)
    ray_inner = s_1.propagate_ray(ray_initial)
    
    intersection_2, normals_2 = s_2.enrich_intersection_geometries(ray_inner)
    output_direction = s_2.scatter_direction_exact(ray_inner)
    ray_output = s_2.propagate_ray(ray_inner)
    
    s_1.plot(ax=ax, label='Back Surface', color='black')
    ray_initial.plot(ax=ax, label='Initial Ray', color='m')
    s_2.plot(ax=ax, label='Front Surface', color='orange')
    ray_inner.plot(ax=ax, label='Inner Ray', color='c')
    ray_output.plot(ax=ax, label='Output Ray', color='r', length=5)
    for i in range(intersection_2.shape[0]):
        ax.plot([intersection_2[i, 0] - normals_2[i, 0]*2, intersection_2[i, 0] + normals_2[i, 0]*2],
                [intersection_2[i, 1] - normals_2[i, 1]*2, intersection_2[i, 1] + normals_2[i, 1]*2],
                'g--', label='Normal Vector' if i == 0 else "")
    for i in range(intersection_2.shape[0]):
        ax.plot(intersection_2[i, 0], intersection_2[i, 1], 'ro', label='Intersection' if i == 0 else "")
    ax.legend()
    plt.axis('equal')
    ax.grid()
    ax.set_title(f"{ray_output.k_vector @ optical_axis}\n{ray_initial.k_vector @ optical_axis}")
    plt.show()

res=2
pi_rounded = np.round(np.pi, res)

# Create sliders
phi_slider = widgets.FloatSlider(
    value=0,
    min=-pi_rounded,
    max=pi_rounded,
    step=10**-res,
    description='phi',
    layout=Layout(width='1500px')
)

theta_slider = widgets.FloatSlider(
    value=0,
    min=-pi_rounded,
    max=pi_rounded,
    step=10**-res,
    description='theta',
    layout=Layout(width='1500px')
)

# Interactive display
ui = widgets.VBox([phi_slider, theta_slider])
out = widgets.interactive_output(
    update_plot,
    {'phi': phi_slider, 'theta': theta_slider}
)

display(ui, out)



Output()